In [1]:
import itertools
import json
import tempfile
from os import path

import invest_ucm_calibration as iuc
import pandas as pd
import rasterio as rio
import swiss_uhi_utils as suhi
import xarray as xr
from affine import Affine
from tqdm import tqdm

from lausanne_greening_scenarios.invest import utils as invest_utils

# register tqdm with pandas to be able to use `progress_apply`
tqdm.pandas()

/home/martibosch/anaconda3/envs/lausanne-greening-scenarios/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
scenario_metrics_filepath = '../data/interim/scenario-metrics.csv'
scenario_lulc_filepath = '../data/interim/scenario-da.nc'
biophysical_table_filepath = '../data/processed/biophysical-table.csv'
ref_et_da_filepath = '../data/interim/invest/ref-et.nc'
t_da_filepath = '../data/processed/tair-ucm.nc'
calibrated_params_filepath = '../data/interim/invest/calibrated-params.json'

In [16]:
scenario_metrics_df = pd.read_csv(scenario_metrics_filepath)
scenario_metrics_df = scenario_metrics_df.groupby(['interaction', 'change_prop']).mean().drop('scenario_run', axis=1).reset_index()

In [47]:
scenario_lulc_da = xr.open_dataarray(scenario_lulc_filepath).mean('scenario_run')

In [43]:
scenario_lulc_da = xr.open_dataarray(scenario_lulc_filepath)

In [48]:
lulc_arr = scenario_lulc_da.sel({
    scenario_dim: row[scenario_dim]
    for scenario_dim in scenario_dims
})

In [42]:
scenario_lulc_da

<xarray.DataArray (interaction: 2, change_prop: 11, y: 1052, x: 2131)>
array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
...
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]]])
Coordinates:
  * interaction  (interaction) object 'cluster' 'scatter'
  * change_prop  (change_prop) float64 0.0 0.1 0.2 0.3 0.4 ... 0.7 0.8 0.9 1.0
  * y            (y) float64 1.159e+06 1.159e+06 ... 1.149e+06 1.149e+06
  * x            (x) float64 2.525e+06 2.525e+06 ... 2.546e+06 2.546e+06

In [40]:
scenario_lulc_da.mean(('scenario_run'))

<xarray.DataArray (interaction: 2, change_prop: 11, y: 1052, x: 2131)>
array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
...
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]]])
Coordinates:
  * interaction  (interaction) object 'cluster' 'scatter'
  * change_prop  (change_prop) float64 0.0 0.1 0.2 0.3 0.4 ... 0.7 0.8 0.9 1.0
  * y            (y) float64 1.159e+06 1.159e+06 ... 1.149e+06 1.149e+06
  * x            (x) float64 2.525e+06 2.525e+06 ... 2.546e+06 2.546e+06

In [24]:
scenario_lulc_da = xr.open_dataarray(scenario_lulc_filepath)
scenario_dims = ('interaction', 'change_prop') # scenario_lulc_da.coords.dims[:3]
# nodata = scenario_lulc_da.attrs['nodata']
change_props = scenario_lulc_da['change_prop'].values
scenario_runs = scenario_lulc_da['scenario_run'].values
rio_meta = scenario_lulc_da.attrs.copy()
rio_meta['transform'] = Affine.from_gdal(*rio_meta['transform'])

with open(calibrated_params_filepath) as src:
    model_params = json.load(src)

In [19]:
t_da = xr.open_dataarray(t_da_filepath)
hottest_day = t_da.isel(time=t_da.groupby('time').max(dim=['x', 'y']).argmax())['time'].dt.strftime('%Y-%m-%d').item()
# hottest_day = '2019-07-24'
t_ref = t_da.sel(time=hottest_day).min(dim=['x', 'y']).item()
uhi_max = t_da.sel(time=hottest_day).max(dim=['x', 'y']).item() - t_ref

ref_et_da = xr.open_dataarray(ref_et_da_filepath).sel(time=hottest_day)
ref_et_raster_dir = '../data/interim'
ref_et_raster_filepath = invest_utils.dump_ref_et_raster(
    ref_et_da, hottest_day, ref_et_raster_dir, invest_utils.get_da_rio_meta(ref_et_da))

In [25]:
# define the functions so that the fixed arguments are curried into them,
# except for `metrics`
def compute_t_avg(row):
    # landscape_arr = sg.generate_landscape_arr(shade_threshold,
    #                                           row['change_prop'],
    #                                           interaction=row['interaction'])
    lulc_arr = scenario_lulc_da.sel({
        scenario_dim: row[scenario_dim]
        for scenario_dim in scenario_dims
    }).values

    with tempfile.TemporaryDirectory() as tmp_dir:
        lulc_raster_filepath = path.join(tmp_dir, 'lulc.tif')
        with rio.open(lulc_raster_filepath, 'w', **rio_meta) as dst:
            dst.write(lulc_arr, 1)

        ucm_wrapper = iuc.UCMWrapper(lulc_raster_filepath, biophysical_table_filepath, 'factors', ref_et_raster_filepath, t_ref, uhi_max, extra_ucm_args=model_params)
        return ucm_wrapper.predict_t_da().mean(skipna=True).item()

In [27]:
row = scenario_metrics_df.iloc[0]

In [28]:
row

interaction                    cluster
change_prop                          0
area_mn                    8.74294e+09
edge_density                0.00076336
shape_index_mn                 1.20635
proportion_of_landscape        17.2642
Name: 0, dtype: object

In [29]:
lulc_arr = scenario_lulc_da.sel({
    scenario_dim: row[scenario_dim]
    for scenario_dim in scenario_dims
}).values


In [31]:
lulc_arr.shape

(10, 1052, 2131)

In [26]:
scenario_metrics_df['T_avg'] = scenario_metrics_df.progress_apply(compute_t_avg, axis=1)

  0%|          | 0/22 [00:00<?, ?it/s]

  5%|▍         | 1/22 [00:00<00:00, 28.66it/s]

ValueError: Source shape (1, 10, 1052, 2131) is inconsistent with given indexes 1